In [1]:
! pip install librosa

In [2]:
# DETECTOR DE COVID POR AUDIO CON DATASET COUGHVID
# David Miguel Garcia Palacios
# Universidad Nacional de Colombia Sede Bogota

# Limpieza de variables
%reset -f

# Instalacion y llamado de librerias
import pandas as pd
import librosa
import os
import warnings

# Ignorar advertencias de Librosa
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Verificacion y descarga del dataset COUGHVID V3 a traves de Zenodo
if os.path.exists("coughvid_20211012"):
  dir = os.path.abspath("coughvid_20211012")
  print(f"El dataset COUGHVID V3 fue detectado en el directorio: {dir}")
else:
  print("Descargando y descomprimiendo dataset COUGHVID V3 a traves de Zenodo")
  !wget {"https://zenodo.org/records/7024894/files/public_dataset_v3.zip"}
  !unzip public_dataset_v3.zip
  !rm public_dataset_v3.zip

# Lectura de metadata del csv
metadata = pd.read_csv("coughvid_20211012/metadata_compiled.csv")

# Obtencion de metadata reducida para pruebas
metadata = metadata.head(3000)

a = metadata.iloc[1,11]

# Obtencion de datos
audio_data = []
label_data = []
for id, row in metadata.iterrows():
  try:
    if isinstance(row["status"],str):
      label = row["status"]
    elif isinstance(row["status_SSL"],str):
      label = row["status_SSL"]
    else:
      raise ValueError("No status found")
    if os.path.exists(f"coughvid_20211012/{row['uuid']}.webm"):
      audio , _ =  librosa.load(f"coughvid_20211012/{row['uuid']}.webm",sr=16000,mono=True)
    else:
      raise ValueError("No file found")
  except: print(f"{id}\t File {row['uuid']} REJECTED")
  else:
    label_data.append(label)
    audio_data.append(audio)
    print(f"{id}\t File {row['uuid']} ACCEPTED")


# Obtencion de los datos de audio
#for uuid in metadata.iloc[:,14]:
#  print(uuid)


El dataset COUGHVID V3 fue detectado en el directorio: /content/coughvid_20211012
0	 File 00014dcc-0f06-4c27-8c7b-737b18a2cf4c REJECTED
1	 File 00039425-7f3a-42aa-ac13-834aaa2b6b92 ACCEPTED
2	 File 0007c6f1-5441-40e6-9aaf-a761d8f2da3b ACCEPTED
3	 File 00098cdb-4da1-4aa7-825a-4f1b9abc214b REJECTED
4	 File 0009eb28-d8be-4dc1-92bb-907e53bc5c7a ACCEPTED
5	 File 0012c608-33d0-4ef7-bde3-75a0b1a0024e REJECTED
6	 File 001328dc-ea5d-4847-9ccf-c5aa2a3f2d0f ACCEPTED
7	 File 00196ba6-0087-484b-a104-3e8884599596 REJECTED
8	 File 001c85a8-cc4d-4921-9297-848be52d4715 REJECTED
9	 File 001d8e33-a4af-4edb-98ba-b03f891d9a6c ACCEPTED
10	 File 001e2f19-d81c-4029-b33c-d2db56b23a4a ACCEPTED
11	 File 00273cdf-ed90-4105-84ec-0c88d52f1dc0 REJECTED
12	 File 0028b68c-aca4-4f4f-bb1d-cb4ed5bbd952 ACCEPTED
13	 File 00291cce-36a0-4a29-9e2d-c1d96ca17242 ACCEPTED
14	 File 0029d048-898a-4c70-89c7-0815cdcf7391 ACCEPTED
15	 File 002d28bc-7806-4dfb-9c9b-afa8cb623cac ACCEPTED
16	 File 002db0bd-e57f-4c30-ade0-16640d424eb7 RE

In [3]:
 from sklearn.preprocessing import LabelEncoder
 from keras.utils import to_categorical

 label_encoder = LabelEncoder()
 integer_encoded = label_encoder.fit_transform(label_data)
 one_hot_encoded = to_categorical(integer_encoded)

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(audio_data, one_hot_encoded, test_size=0.2, random_state=42)

In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train_padded = pad_sequences(X_train, dtype='float32', padding='post')
X_val_padded = pad_sequences(X_val, padding='post', maxlen=X_train_padded.shape[1], dtype='float32')

# Define the model
model = models.Sequential()
model.add(layers.Dense(128, activation='relu', input_shape=(X_train_padded.shape[1],)))  # Input shape is the length of the padded sequences
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))  # 3 classes for multi-class classification

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary (optional)
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │      47,431,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 47,440,259 (180.97 MB)

 Trainable params: 47,440,259 (180.97 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
print("Shape of X_train_padded:", X_train_padded.shape)  # Should be (212, num_features)
print("Shape of y_train:", y_train.shape)                # Should be (212, 3)

# Train the model
history = model.fit(X_train_padded, y_train, validation_split=0.2, epochs=50, batch_size=500)


Shape of X_train_padded: (1291, 370560)
Shape of y_train: (1291, 3)
Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 13s 4s/step - accuracy: 0.4729 - loss: 1.0839 - val_accuracy: 0.6448 - val_loss: 1.0223
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.8994 - loss: 0.6123 - val_accuracy: 0.7104 - val_loss: 0.9479
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.9082 - loss: 0.4451 - val_accuracy: 0.7375 - val_loss: 0.8952
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 13s 3s/step - accuracy: 0.9419 - loss: 0.3186 - val_accuracy: 0.7336 - val_loss: 0.8689
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - accuracy: 0.9518 - loss: 0.2643 - val_accuracy: 0.7220 - val_loss: 0.8584
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.9537 - loss: 0.2186 - val_accuracy: 0.7181 - val_loss: 0.8565
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 12s 4s/step - accuracy: 0.9700 - loss: 0.1755 - val_accuracy: 0.7066 - val_loss: 0.8597
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 3s/step - accuracy: 0.968

In [8]:
#X_val_padded = pad_sequences(X_val, padding='post', maxlen=X_train_padded.shape[1], dtype='float32')

# Check the shape of X_val_padded
print("Shape of X_val_padded:", X_val_padded.shape)
print("Shape of y_train:", y_val.shape)

model.evaluate(X_val_padded, y_val)
model.save('modelo_covid_detector.keras')


Shape of X_val_padded: (323, 370560)
Shape of y_train: (323, 3)
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 252ms/step - accuracy: 0.6508 - loss: 1.1104
